In [84]:
import pandas as pd
from plotly import express as px
import json
import nbformat
import tiktoken
import os
from dotenv import load_dotenv

# Exercício 4:

In [4]:
df = pd.read_json('data\manchetes_categorizadas.json')

df.head()

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Matheus\AppData\Local\Temp\ipykernel_27312\1120884369.py:1: SyntaxWarning: invalid escape sequence '\m'
  df = pd.read_json('data\manchetes_categorizadas.json')


,manchete,categoria
0,"O Brasil atingiu a marca de 62,5 milhões de co...",Neutra
1,O valor médio de cada dívida dos brasileiros c...,Neutra
2,"Entre os brasileiros inadimplentes, cada um de...",Neutra
3,Dívidas com bancos e cartões de crédito aparec...,Neutra
4,O valor total das dívidas dos brasileiros inad...,Neutra


In [85]:
df_grouped = df.groupby('categoria', as_index=False).count()

fig = px.bar(df_grouped,
            x = 'manchete',
            y = 'categoria',
            text_auto=True,
            title='Quantidade de Manchetes de Inadimplência Classificadas por Categoria'
            )

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'manchete=%{x}<br>categoria=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'h',
              'showlegend': False,
              'textposition': 'auto',
              'texttemplate': '%{x}',
              'type': 'bar',
              'x': array([ 11, 112,   1]),
              'xaxis': 'x',
              'y': array(['Negativa', 'Neutra', 'Positiva'], dtype=object),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Quantidade de Manchetes de Inadimplência Classificadas por Categoria'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'manchete'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'categoria'}}}
})

# Exercício 5:

In [24]:
import pandas as pd

df_script = pd.read_csv('data/simpsons_script_lines.csv', low_memory=False)
df_episodes = pd.read_csv('data/simpsons_episodes.csv', low_memory=False)
df_characters = pd.read_csv('data/simpsons_characters.csv', low_memory=False)
df_locations = pd.read_csv('data/simpsons_locations.csv', low_memory=False)

df_script.set_index('id', inplace=True)
df_characters['id'] = df_characters['id'].astype(str)

df_characters = df_characters.add_prefix('character_')
df_locations = df_locations.add_prefix('location_')
df_episodes = df_episodes.add_prefix('episode_')

data = (
    df_script.merge(df_episodes, left_on='episode_id', right_on='episode_id')
             .merge(df_characters, left_on='character_id', right_on='character_id', how='left')
             .merge(df_locations, left_on='location_id', right_on='location_id', how='left')
)


assert data.shape[0] == df_script.shape[0]

In [26]:
data.head()

,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,...,episode_season,episode_title,episode_us_viewers_in_millions,episode_video_url,episode_views,character_name,character_normalized_name,character_gender,location_name,location_normalized_name
0,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,true,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",...,2,Lisa's Substitute,17.7,http://www.simpsonsworld.com/video/288011331912,52770.0,Miss Hoover,miss hoover,f,Springfield Elementary School,springfield elementary school
1,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,true,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,...,2,Lisa's Substitute,17.7,http://www.simpsonsworld.com/video/288011331912,52770.0,Lisa Simpson,lisa simpson,f,Springfield Elementary School,springfield elementary school
2,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,true,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,...,2,Lisa's Substitute,17.7,http://www.simpsonsworld.com/video/288011331912,52770.0,Miss Hoover,miss hoover,f,Springfield Elementary School,springfield elementary school
3,32,212,Lisa Simpson: That life is worth living.,864000,true,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,...,2,Lisa's Substitute,17.7,http://www.simpsonsworld.com/video/288011331912,52770.0,Lisa Simpson,lisa simpson,f,Springfield Elementary School,springfield elementary school
4,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,true,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,...,2,Lisa's Substitute,17.7,http://www.simpsonsworld.com/video/288011331912,52770.0,Edna Krabappel-Flanders,edna krabappel-flanders,f,Springfield Elementary School,springfield elementary school


In [28]:
import tiktoken

# Função para estimar número de tokens
def estimar_tokens(texto):
    encoder = tiktoken.get_encoding("cl100k_base")  # Exemplo de codificação
    # encoder = tiktoken.get_encoding("gpt-4o")  # Exemplo de codificação
    tokens = encoder.encode(texto)
    return tokens

In [46]:
X = data.dropna(subset='normalized_text').copy()
X['n_tokens'] = X.normalized_text.fillna('').apply(lambda x: len(estimar_tokens(x)))
print(X.shape)

print(X['episode_season'])

(132087, 31)
0         2
1         2
2         2
3         2
4         2
         ..
158266    2
158267    2
158268    2
158269    2
158270    2
Name: episode_season, Length: 132087, dtype: int64


## Média por episódio

In [41]:
episodios_tokens = X.groupby('episode_id', as_index=False)['n_tokens'].sum()
episodios_tokens['n_tokens'].mean()
print(f'A quantidade média de tokens por episódio é de {episodios_tokens['n_tokens'].mean():.0f} tokens')

A quantidade média de tokens por episódio é de 2607 tokens


## Média por Temporada

In [48]:
temporadas_tokens = X.groupby('episode_season', as_index=False)['n_tokens'].sum()
temporadas_tokens['n_tokens'].mean()
print(f'A quantidade média de tokens por temporada é de {temporadas_tokens['n_tokens'].mean():.0f} tokens')

A quantidade média de tokens por temporada é de 56551 tokens


## Temporada com mais Tokens

In [68]:
temporadas_tokens = X.groupby('episode_season', as_index=False)['n_tokens'].sum()
temporada_max_token = temporadas_tokens[temporadas_tokens['n_tokens'] == temporadas_tokens['n_tokens'].max()]
temporada = temporada_max_token['episode_season']
tokens_seas = temporada_max_token['n_tokens']
print(f'A temporada que teve mais tokens foi {temporada}, com um total de {tokens} tokens.')

A temporada que teve mais tokens foi 6    7
Name: episode_season, dtype: int64, com um total de 48    3454
Name: n_tokens, dtype: int64 tokens.


## Episódio com mais tokens

In [80]:
episodios_tokens = X.groupby('episode_id', as_index=False)['n_tokens'].sum()
episodio_max_token = episodios_tokens[episodios_tokens['n_tokens'] == episodios_tokens['n_tokens'].max()]
episodio = episodio_max_token['episode_id']
tokens_epis = episodio_max_token['n_tokens']
print(f'A episodio que teve mais tokens foi {episodio}, com um total de {tokens_epis} tokens.')

A episodio que teve mais tokens foi 48    49
Name: episode_id, dtype: int64, com um total de 48    3454
Name: n_tokens, dtype: int64 tokens.


# Exercício 6

In [89]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

examples = {
    'Positive': 'Hey, thanks for your vote, man',
    'Neutral': 'That was our wedding',
    'Negative': 'Oh, please',
}

class ChunkClassificator():
    def __init__(self, model_name, apikey, text, window_size, overlap_size):
        self.text = text
        if isinstance(self.text, str):
            self.text = [self.text]
        self.window_size = window_size
        self.overlap_size = overlap_size
        # Aplicacao dos chunks
        self.chunks = self.__text_to_chunks()
        self.model = self.__create_model(apikey, model_name)


    def __create_model(self, apikey, model_name):
        genai.configure(api_key=apikey)
        self.prompt_base = f"""
        You are a classificator from the "The Simpsons" show.
        You will receive the #conversations# from real episodes in the format:
        <location>, <character> said: <character line>
        
        Analyze the following excerpts of dialogue from The Simpsons and classify the sentiment of each as Positive, Neutral, or Negative.

        Consider the tone, context, and emotional undertones of the text to determine the appropriate category:

        Positive: Expresses happiness, optimism, humor, or an overall positive attitude.
        Neutral: Is factual, ambiguous, or lacks strong emotional content.
        Negative: Conveys sadness, anger, frustration, or any negative sentiment.
        For each excerpt, respond with the sentiment category (Positive, Neutral, or Negative)

        Check this examples to help you to define the category:
        {examples}

        """
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
        generation_config = {
            'temperature': 0.2,
            'top_p': 0.8,
            'top_k': 20,
            'max_output_tokens': 1000
        }
        return genai.GenerativeModel(
            model_name,
            system_instruction=self.prompt_base,
            generation_config = generation_config,
            safety_settings=safety_settings
        )


    
    def __text_to_chunks(self):       
        n = self.window_size  # Tamanho de cada chunk
        m = self.overlap_size  # overlap entre chunks
        return [self.text[i:i+n] for i in range(0, len(self.text), n-m)]


    def __create_chunk_prompt(self, chunk):
        episode_lines = '\n'.join(chunk)
        prompt = f"""
        #subtitles#
        {episode_lines}
        ######
        Summarize it.
        """
        return prompt
        
    
    def __summarize_chunks(self):
        # Loop over chunks
        chunk_summaries = []
        for i, chunk in enumerate(self.chunks):
            print(f'Summarizing chunk {i+1} from {len(self.chunks)}')
            # Create prompt
            prompt = self.__create_chunk_prompt(chunk)
            response = self.model.generate_content(prompt)
            # Apendar resposta do chunk
            chunk_summaries.append(response.text)
            
            # if i == 4: break

        return chunk_summaries


    def summarize(self):
        print('Summarizing text')
        # Chamar o sumario dos chunks
        self.chunk_summaries = self.__summarize_chunks()
        # Prompt final
        summaries = '- ' + '\n- '.join(self.chunk_summaries)
        prompt = f"""
        You are a classificator from the "The Simpsons" show.
        Analyze the following excerpts of dialogue from The Simpsons and classify the sentiment of each as Positive, Neutral, or Negative.

        Consider the tone, context, and emotional undertones of the text to determine the appropriate category:

        Positive: Expresses happiness, optimism, humor, or an overall positive attitude.
        Neutral: Is factual, ambiguous, or lacks strong emotional content.
        Negative: Conveys sadness, anger, frustration, or any negative sentiment.
        For each excerpt, respond with the sentiment category (Positive, Neutral, or Negative)
        {summaries}
        ######
        """
        print('Final summarization')
        response = self.model.generate_content(prompt)
        
        return response.text

In [97]:
episode_season = 5
episode_id = 92
X = (data[(data.episode_season == episode_season) &
          (data.episode_id == episode_id)].sort_values('number')
)

X['line'] = (X['location_normalized_name'].fillna('') + ', ' + 
             X['character_normalized_name'].fillna('') + ' said: ' + 
             X['normalized_text'].fillna('')
)

import os
from dotenv import load_dotenv
load_dotenv('../.env')

summarizer = ChunkClassificator(
    model_name = "gemini-1.5-flash",
    apikey = os.environ["GEMINI_API"],
    text = X['line'].tolist(),
    window_size = 40,
    overlap_size = 5
)

episode_summary = summarizer.summarize()

KeyError: 'GEMINI_API'